In [14]:
# Processing E files :
import sys
import re
from datetime import datetime, timedelta
import scipy

In [3]:
recDict = dict()
commentDict = dict()
delayDict = dict()
dbarrayDict = dict()
minPts = 200

In [5]:
def cleanFields(comments):
    comments = comments.replace(',', '')
    fields = ['d+' if items.isdigit() else items for items in comments.split()]
    newFields = ['V*' if '=' in items else items for items in fields ]
    newFields2 = ' '.join(['V*' if items != 'd+' and not items.isalpha() else items for items in newFields ])
    return newFields2


In [6]:
def getGID(cleanedComments, subcomp, severity):
    if cleanedComments in commentDict.keys():
        return commentDict[cleanedComments]
    else:
        gidVal = len(commentDict) + 1
        commentDict[cleanedComments] = gidVal
        print cleanedComments, gidVal
        return gidVal


In [7]:
def generateDelays():
    for i in reversed(range(len(recDict))):
        recA = recDict[i+1]
        if recA[2] == 'FATAL' or recA[2] == 'ERROR':
            curr_time = datetime.strptime(recA[3],'%Y-%m-%d-%H.%M.%S.%f')
            j = i
            while True:
                if j == 0:
                    break
                else:
                    recB = recDict[j]
                    prev_time = datetime.strptime(recB[3],'%Y-%m-%d-%H.%M.%S.%f')
                    if (curr_time - prev_time) < timedelta(days=2):
                        if (recA[4],recB[4]) in delayDict:
                            l = delayDict[(recA[4], recB[4])]
                            l.append(curr_time -prev_time)
                            delayDict[(recA[4], recB[4])] = l
                        else:
                            delayDict[(recA[4], recB[4])] = []
                            delayDict[(recA[4], recB[4])].append(curr_time - prev_time)
                        j -= 1
                    else:
                        break
    #
    for keys in delayDict:
        dbarrayDict[keys] = len(delayDict[keys])

In [8]:
# It is a random algorithm, so there is no real trouble if we use one way or the other.
#---------------------------------------------------------------------
def runDBScan():
    CG = []
    for keys in dbarrayDict:
        if dbarrayDict[keys] > minPts:
            CG.append((keys,delayDict[keys][0]))
    for items in range(len(CG)):
        print CG[items]

In [15]:
inputFile = 'Intrepid_RAS_0901_0908_scrubbed'
count = 1

with open(inputFile) as fp:
    for newLine in fp:
        if len(newLine) > 323:
            comments = newLine[323:].strip()
            index = 322
            if not newLine[index].isspace():
                while not newLine[index].isspace() and not newLine[index+1].isspace():
                    index -= 1
                comments = newLine[index:].strip()

        data = newLine.strip().split()
        print comments
        if len(data) > 0 and data[0].startswith('2'):
            cleanedComments = cleanFields(comments)
            newData = [data[items] for items in range(len(data)) if items < 12 and len(data[items]) > 1][3:7]
            gid = getGID(cleanedComments, newData[1], newData[2])
            newData.append(gid)
            recDict[count] = newData
            count += 1
            if count > 10000:
                break


L3 Correctable error L3 controller=0x00000000  group=0x00000007  status=0x0000000B  errstat=0x40000000  dir_ce_count=0x00006D30  dir_ce_addr=0x00D6F4C0  edr_ce_count=0x00000000  edr_ce_addr=0x00000000
L3 Correctable error L3 controller=0x00000000  group=0x00000007  status=0x0000000B  errstat=0x40000000  dir_ce_count=0x00006D30  dir_ce_addr=0x00D6F4C0  edr_ce_count=0x00000000  edr_ce_addr=0x00000000
MESSAGE
MESSAGE
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------